In [3]:
import json, re
from collections import defaultdict
import pandas as pd

In [4]:
import glob

# Get list of TSV files in folder
tsv_files = glob.glob("./eval_res/*.txt")

# print(tsv_files)
all_res = defaultdict(dict)
task_metric = defaultdict(list)

all_res_std = defaultdict(dict)


In [5]:

for file in tsv_files:
    results = json.load(open(file))
    model = results["config"]["model_args"].replace("pretrained=", "").split("/")[-1]
    if "wrapper" in file:
        model = model + "-wrapper"
    task = list(results["results"].keys())[0]
    
    res_keys = sorted(list(results["results"][task].keys()), key=lambda x: len(x))
    metric = 0.0
    metric_std = 0.0
    metric_name = None
    
    for key in res_keys:
        if "std" not in key:
            metric = results["results"][task][key] * 100
            metric_name = key
        else:
            metric_std = results["results"][task][key] * 100
            
    if task == "record":
        metric = results["results"][task]["f1"] * 100
        metric_std = results["results"][task]["f1_stderr"] * 100
        metric_name = "f1"
    
    all_res[model][task] = metric
    all_res_std[model][task] = metric_std
    task_metric[task].append(metric_name) 
    


In [6]:
df = pd.DataFrame(all_res_std)
df.columns

Index(['flan-t5-small-distil-v2', 'Cerebras-GPT-256M-distil-v1',
       't5-large-distil-v1', 't5-base', 'gpt2', 't5-base-distil-v2',
       'Cerebras-GPT-111M', 'gpt2-xl', 'Cerebras-GPT-1.3B-distil-v1-wrapper',
       'Cerebras-GPT-590M-distil-v1-wrapper', 'gpt2-large', 'flan-t5-base',
       't5-large', 'Cerebras-GPT-111M-distil-v1-wrapper', 't5-small',
       'flan-t5-base-distil-v4', 'gpt-neo-125m', 'gpt2-xl-distil-v1-wrapper',
       'Cerebras-GPT-256M', 'gpt2-distil-v1-wrapper',
       'Cerebras-GPT-590M-distil-v1', 'flan-t5-large', 'flan-t5-small',
       'Cerebras-GPT-256M-distil-v1-wrapper', 'gpt2-large-distil-v1-wrapper',
       'Cerebras-GPT-1.3B', 'Cerebras-GPT-590M', 'gpt-neo-1.3B',
       'flan-t5-large-distil-v1', 'gpt-neo-125m-distil-v1-wrapper'],
      dtype='object')

In [8]:
sorted_columns = ["t5-small", 'flan-t5-small', 'flan-t5-small-distil-v2', 
                't5-base', 'flan-t5-base', 'flan-t5-base-distil-v4',
                'Cerebras-GPT-256M', 'Cerebras-GPT-256M-distil-v1', 'Cerebras-GPT-256M-distil-v1-wrapper',
                'Cerebras-GPT-590M', 'Cerebras-GPT-590M-distil-v1', 'Cerebras-GPT-590M-distil-v1-wrapper',
                'Cerebras-GPT-111M', 'Cerebras-GPT-111M-distil-v1-wrapper',
                'gpt-neo-125m','gpt-neo-125m-distil-v1-wrapper',
                'gpt2', 'gpt2-distil-v1-wrapper',
                'gpt2-large', 'gpt2-large-distil-v1-wrapper',
                'gpt2-xl', 'gpt2-xl-distil-v1-wrapper', 
                't5-large', 'flan-t5-large', 'flan-t5-large-distil-v1', 
                'Cerebras-GPT-1.3B', 'gpt-neo-1.3B', 't5-large-distil-v1',
                't5-base-distil-v2',
                'Cerebras-GPT-1.3B-distil-v1-wrapper',
                ]

task_sort = ["openbookqa", "sciq", "race", "arc_challenge", "piqa",
            
            "record", 
            
            "webqs", #"triviaqa",
            
            "sst", 
            
            "mrpc", #"qqp", 
            
            "cb", "rte", "mnli",  "mnli_mismatched",
            
            "wsc", "wsc273", "winogrande",
            
            "wic", 
            
            "copa", "hellaswag"]

In [9]:
[x for x in list(df.columns) if x not in sorted_columns ]

[]

### metric

In [10]:
df = pd.DataFrame(all_res)

df = df[sorted_columns]

sorted_df = df.loc[task_sort]

metrics = [task_metric[x][0] for x in task_sort]

sorted_df.insert(loc=0, column="metric", value=metrics)
sorted_df.to_csv("metric_res.csv")

### std

In [7]:
df = pd.DataFrame(all_res_std)

df = df[sorted_columns]

sorted_df = df.loc[task_sort]

metrics = [task_metric[x][0] for x in task_sort]

sorted_df.insert(loc=0, column="metric", value=metrics)
sorted_df.to_csv("metric_std.csv")

In [8]:
sorted_df

,metric,t5-small,flan-t5-small,flan-t5-small-distil-v2,t5-base,flan-t5-base,flan-t5-base-distil-v4,Cerebras-GPT-256M,Cerebras-GPT-256M-distil-v1,Cerebras-GPT-256M-distil-v1-wrapper,...,gpt2-xl,gpt2-xl-distil-v1-wrapper,t5-large,flan-t5-large,Cerebras-GPT-1.3B,gpt-neo-1.3B,t5-large-distil-v1,t5-base-distil-v2,Cerebras-GPT-1.3B-distil-v1-wrapper,flan-t5-large-distil-v1
openbookqa,acc_norm,30.200000,27.000000,30.000000,34.800000,28.800000,32.800000,25.400000,28.600000,30.600000,...,32.000000,39.800000,32.800000,31.200000,29.000000,33.600000,36.000000,32.000000,34.000000,34.000000
sciq,acc_norm,58.000000,89.000000,79.400000,71.700000,93.000000,86.500000,65.700000,70.400000,68.800000,...,76.100000,80.400000,82.400000,93.800000,73.000000,77.100000,84.500000,82.900000,79.400000,86.700000
race,acc,26.411483,29.665072,28.899522,31.100478,35.885167,33.971292,27.464115,29.186603,27.081340,...,33.110048,39.138756,31.483254,40.861244,30.334928,34.066986,32.631579,32.631579,32.918660,32.822967
arc_challenge,acc_norm,22.696246,22.269625,23.976109,24.402730,25.085324,27.815700,21.928328,24.232082,26.109215,...,28.498294,35.750853,25.426621,30.716724,25.255973,25.853242,29.010239,26.450512,30.290102,31.825939
piqa,acc_norm,55.331882,61.860718,61.860718,55.712731,66.974973,65.778020,61.425462,61.479869,61.425462,...,70.511425,71.327530,55.930359,72.198041,66.757345,71.055495,67.192601,63.982590,66.866159,70.565832
record,f1,53.401905,57.705238,53.752524,64.551667,68.232905,61.513333,61.206714,59.407524,58.568333,...,84.449238,78.495952,73.068524,76.698143,75.027952,81.377476,68.723429,59.097095,66.331797,70.401762
webqs,acc,0.000000,7.874016,0.000000,0.000000,16.289370,0.000000,0.000000,1.525591,0.000000,...,1.574803,0.000000,0.000000,22.588583,0.935039,1.673228,0.000000,0.000000,0.000000,0.000000
sst,acc,70.986239,87.270642,85.665138,57.339450,92.316514,91.399083,49.770642,58.256881,76.949541,...,49.082569,93.463303,50.229358,94.036697,51.261468,65.711009,90.252294,91.169725,90.252294,93.119266
mrpc,acc,48.039216,63.235294,58.578431,31.617647,71.323529,74.754902,68.382353,68.382353,68.382353,...,63.235294,75.980392,34.313725,82.598039,68.382353,68.382353,71.078431,73.529412,71.323529,77.941176
cb,acc,41.071429,42.857143,50.000000,64.285714,82.142857,58.928571,44.642857,7.142857,7.142857,...,39.285714,64.285714,75.000000,82.142857,30.357143,44.642857,51.785714,30.357143,28.571429,66.071429
